# Preparation

In [1]:
import os
from hanlp_restful import HanLPClient
import pandas as pd
from docx import Document
from tqdm import tqdm
import ast


In [2]:
# file_list = os.listdir("data/Interview transcripts-round 3")
file_list = ['1_SZUZJY_来深33年邹先生.docx',
 '1_SZUZJY_深圳本地人林女士.docx',
 '2_JNU LLW_DB.docx',
 '2_JNU LLW_LXS.docx',
 '2_JNU LLW_RRAY.docx',
 '2_JNU LLW_SYJZ.docx',
 '2_JNU LLW_XBB.docx',
 '3_SZTUWWX_李先生.docx',
 '3_SZTUWWX_许女士.docx',
 '3_SZTUWWX_谭女士霍先生.docx',
 '3_SZTUWWX_邓先生王女士.docx',
 '3_SZTUWWX_陈女士.docx',
 '4_UT_LJY_UU.docx',
 '4_UT_LJY_WW.docx',
 '4_UT_LJY_XW.docx',
 '5_CS+刘阿姨.docx',
 '5_CS+尤老师.docx',
 '5_CS+钟老师.docx',
 '5_CS+黄先生.docx',
 '6_szulys_nw.docx',
 '6_szulys_乔麦.docx',
 '7_SZUF_LT.docx',
 '7_SZUF_坚.docx',
 '7_SZUF_大花.docx',
 '8_hkust_wdq_q.docx',
 '8_hkust_wdq_yyq.docx',
 '9_SZUCJH_发哥.docx',
 '9_SZUCJH_小贾.docx',
 '9_SZUCJH_小陈.docx',
 '9_SZUCJH_老陈.docx',
 '10_SZTU-lx_何爷爷.docx',
 '10_SZTU-lx_凹凸曼&李先生.docx',
 '11_SZUH_LEI.docx',
 '11_SZUH_YUNWEI.docx',
 '12_SZUL_Roson.docx',
 '12_SZUL_夏天.docx',
 '12_SZUL_张律师.docx',
 '12_SZUL_美丽.docx',
 '12_SZUL_花生.docx',
 '13_SZTULJK_HSF.docx',
 '13_SZTULJK_HYZ.docx',
 '13_SZTULJK_LDH.docx',
 '13_SZTULJK_LJP.docx',
 '14_SZUlsr-jiachunxia.docx',
 '14_SZUlsr-wuxin.docx',
 '15_PKU+DYE_Suning.docx',
 '15_PKU+DYE_Xin.docx',
 '15_PKU+DYE_Xuan.docx',
 '17_SCNUC_小于先生.docx',
 '17_SCNUC_小于女士.docx',
 '18_SZU_XZA_HE.docx',
 '19_sustech_qk_He.docx',
 '19_sustech_qk_meng.docx',
]

# Hanlp

In [3]:
# HanLP = HanLPClient('https://www.hanlp.com/api', auth='你申请到的auth')  # auth需要申请
HanLP = HanLPClient('https://www.hanlp.com/api', auth="NTc3MkBiYnMuaGFubHAuY29tOnV6R0xMS05pblB3c29CZE4=", language='zh')

In [4]:
# tokenize(text: Union[str, List[str]], coarse: Optional[bool] = None, language=None) → List[List[str]]
# seg = HanLP.tokenize(text)
# seg

In [5]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return ''.join(full_text)

# def get_location_from_one_hanlp(my_filepath):
#     text = read_docx(my_filepath)
#     # display(text)

#     doc = HanLP(text, tasks='ner/pku', language='zh')
#     my_ner_list = doc['ner/pku']

#     my_filename = my_filepath.split("/")[-1].split(".")[0]
#     return my_ner_list, my_filename

def split_text(text, max_length=15000):
    # 使用\n分割文本为句子列表
    sentences = text.split('\n')
    # display(len(sentences))
    parts = []
    current_part = ""

    for sentence in sentences:
        # 检查加入当前句子是否会超过最大长度
        if len(current_part) + len(sentence) + 1 > max_length:  # +1 是为了计算加入的换行符
            # 如果会超过，当前部分结束，开始新的部分
            parts.append(current_part)
            current_part = sentence  # 开始新的部分，当前句子是新部分的第一句
        else:
            # 如果不会超过，将当前句子加入当前部分
            if current_part:  # 如果当前部分不为空，先加入换行符
                current_part += '\n'
            current_part += sentence

    # 循环结束后，将最后一部分（如果有）加入到部分列表中
    if current_part:
        parts.append(current_part)

    return parts

def get_location_from_one_hanlp(my_filepath):
    text = read_docx(my_filepath)
    # print(text)

    # 检查文本长度并分割
    # display(len(text.split("\n")))
    texts = split_text(text) if len(text) > 15000 else [text]
    # 初始化空列表来存储所有部分的结果
    all_ner_lists = []
    # display(len(texts))
    # display(len(texts[0].split("\n")))

    # 对每个文本部分调用 API 并合并结果
    for part_text in texts:
        doc = HanLP(part_text, tasks='ner/pku', language='zh')
        
        # doc.pretty_print()
        
        my_ner_list = doc['ner/pku']
        # display(len(my_ner_list))
        all_ner_lists.extend(my_ner_list)

    my_filename = my_filepath.split("/")[-1].split(".")[0]
    # display(len(all_ner_lists))
    return all_ner_lists, my_filename

In [6]:
# doc.pretty_print()

In [7]:
def get_text_length(my_filepath):
    text = read_docx(my_filepath)
    # print(len(text))
    return len(text)

In [8]:
# file_path = "data/Interview transcripts-round 3/1_SZUZJY_来深33年邹先生.docx"
# range_list = [0, 2, 7, 12, 15, 19, 22, 27, 29, 31, 33, 35, 40, 45, 48, 51]
# range_list = range(0, 55)
# # range_list = [1]
# for i in tqdm(range_list):
#     my_filepath = f"data/Interview transcripts-round 3/{file_list[i]}"
#     # tmp = split_text(read_docx(my_filepath))
#     ner_list, filename = get_location_from_one_hanlp(my_filepath)
#     # display(ner_list)
#     with open(f"data/output_hanlp+worldcloud/txt/{filename}.txt","w") as f:
#         f.write("[\n")
#         for item in ner_list:
#             f.write("%s\n" % item)
#         f.write("]")

# Combine `hanlp_list` & `match_list` to df

In [9]:
def combine_list_todf(my_filename):
    result = get_one_hanlp_list(my_filename) + get_one_match_list(my_filename)
    # df = pd.DataFrame(result, columns=['location_name', 'value', 'method']).groupby("location_name").agg({"value": lambda x: list(x)}).reset_index()
    # df["count"] = df["value"].apply(lambda x: len(x))
    df = pd.DataFrame(result, columns=['location_name', 'value', 'method'])
    df["filename"] = my_filename
    df.to_csv(f"data/output_hanlp_match/csv/{my_filename}.csv", index=False,encoding="utf-16",sep="\t")
    return df

def get_one_hanlp_list(my_filename):
    # with open(f"data/output_hanlp/{my_filename}.txt", "r") as f:
    with open(f"data/output_hanlp+worldcloud/txt/{my_filename}.txt", "r") as f:
        content = f.read()
        tmp_ner_list = [ast.literal_eval(i.strip()) for i in content[1:-1].strip().split("\n")]
        result =[[item[0], (index, item[2],item[3]),"hanlp"] for index, sublist in enumerate(tmp_ner_list) for item in sublist if item[1] == 'ns']
    return result

def get_one_match_list(my_filename):
    with open(f"data/output_match/txt/{my_filename}.txt", "r") as f:
        content = f.read()
        tmp_ner_list = [ast.literal_eval(i.strip()) for i in content[1:-1].strip().split("\n")]
        result =[[item[0], (index, item[2],item[3]),"patch"] for index, sublist in enumerate(tmp_ner_list) for item in sublist if item[1] == 'ns']
    return result

In [10]:
df_tmp = combine_list_todf("1_SZUZJY_来深33年邹先生")
df_tmp

,location_name,value,method,filename
0,深圳,"(1, 2, 3)",hanlp,1_SZUZJY_来深33年邹先生
1,深圳,"(1, 30, 31)",hanlp,1_SZUZJY_来深33年邹先生
2,深圳,"(6, 10, 11)",hanlp,1_SZUZJY_来深33年邹先生
3,龙华工业区,"(8, 12, 14)",hanlp,1_SZUZJY_来深33年邹先生
4,罗湖书城,"(8, 33, 35)",hanlp,1_SZUZJY_来深33年邹先生
...,...,...,...,...
259,机场,"(133, 0, 0)",patch,1_SZUZJY_来深33年邹先生
260,11号线,"(134, 0, 0)",patch,1_SZUZJY_来深33年邹先生
261,1号线,"(134, 0, 0)",patch,1_SZUZJY_来深33年邹先生
262,邮局,"(170, 0, 0)",patch,1_SZUZJY_来深33年邹先生


In [11]:
range_list = range(len(file_list))

for i in tqdm(range_list):
    filename = file_list[i].split(".")[0]
    combine_list_todf(filename)

100%|██████████| 53/53 [00:00<00:00, 55.44it/s]
